In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from textblob import TextBlob

# Load Excel file (adjust path as needed)
file_path = "YOUR_PATH/In_Treatment_Session 1.xlsx"
df = pd.read_excel(file_path)

# Convert time format (e.g., 00:01:01,429 → seconds)
def convert_time_to_seconds(time_str):
    h, m, s = time_str.split(":")
    s, ms = s.split(",")
    return int(h) * 3600 + int(m) * 60 + int(s) + int(ms) / 1000

# Filter Laura's lines
df_laura = df[df["Speaker"] == "Laura"].copy()
df_laura["Time (Seconds)"] = df_laura["Start Time"].apply(convert_time_to_seconds)

# Sentiment analysis using TextBlob
df_laura["Sentiment Score"] = df_laura["Text"].apply(lambda text: TextBlob(str(text)).sentiment.polarity)

# 📌 Step 1: Plot sentiment changes over time
plt.figure(figsize=(10, 5))
plt.plot(df_laura["Time (Seconds)"], df_laura["Sentiment Score"], marker="o", linestyle="-")
plt.title("Laura's Sentiment Over Time", fontsize=14)
plt.xlabel("Time (Seconds)", fontsize=12)
plt.ylabel("Sentiment Score", fontsize=12)
plt.axhline(0, color="gray", linestyle="--", linewidth=1)  # baseline
plt.grid(True)
plt.show()

# 📌 Step 2: Count emotional keywords
emotion_categories = {
    "Sadness": ["sad", "depressed", "unhappy", "down", "miserable", "hopeless", 
                "lonely", "crying", "heartbroken", "upset", "devastated"],
    "Anger": ["angry", "mad", "furious", "annoyed", "frustrated", 
              "irritated", "enraged", "resentful", "offended", "hostile"],
    "Fear": ["scared", "afraid", "anxious", "nervous", "worried", 
             "stressed", "uneasy", "panicked", "shaky", "trembling"],
    "Joy": ["happy", "joyful", "excited", "relieved", "grateful", 
            "content", "satisfied", "cheerful", "elated", "hopeful"],
    "Disgust": ["disgusted", "gross", "repulsed", "nauseated", "appalled", 
                "offended", "sickened", "revolted", "horrified", "detestable"],
}

# Initialize counts
emotion_counts = {key: 0 for key in emotion_categories.keys()}

# Count occurrences of emotional words
for text in df_laura["Text"].str.lower():
    for emotion, words in emotion_categories.items():
        emotion_counts[emotion] += sum(1 for word in words if word in text.split())

# 📌 Step 3: Plot emotion word frequency
plt.figure(figsize=(8, 5))
plt.bar(emotion_counts.keys(), emotion_counts.values(), color=["blue", "red", "orange", "green", "purple"])
plt.title("Emotion Word Frequency in Laura's Dialogue", fontsize=14)
plt.xlabel("Emotion Category", fontsize=12)
plt.ylabel("Word Count", fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()


In [ ]:
# Show lines with the lowest possible sentiment
df_laura[df_laura["Sentiment Score"] == -1][["Time (Seconds)", "Text"]]

# Show lines with strong positive sentiment
df_laura[df_laura["Sentiment Score"] > 0.5][["Time (Seconds)", "Text"]]

# Show lines after a specific time threshold (e.g., end of session)
df_laura[df_laura["Time (Seconds)"] > 790][["Time (Seconds)", "Text"]]
